<a href="https://colab.research.google.com/github/BruceAuHK/Dashboard_FFO/blob/master/Plotly_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import urllib.request
import json
import ssl
import plotly as py
import plotly.graph_objects as go
import ipywidgets as widgets
import numpy as np
from scipy import special 
import pandas as pd
from pandas.io.json import json_normalize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

context = ssl._create_unverified_context()

def get_data_from_HKMA(url):
    # Balance sheet
    url = url

    # retrieve data from hkma api, 100 entries per call, untill every data are retrieveed
    records = []
    offset = 0
    while True:
        with urllib.request.urlopen(url+"&offset="+str(offset), context=context) as req:
            data = json.loads(req.read().decode())
            records += data['result']['records']
        if data['result']['datasize'] <= 0:
            break
        else:
            offset += 100

    # now data retrieved from hkma are stored in 'records' in a chronological order
    return records


print("dependencies ready")


dependencies ready


# **Monetary Base**

In [2]:
MBC = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/monetary-operation/monetary-base-endperiod?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. Size of the Monetary Base was published since 25 November 1998.
mbc_df = pd.json_normalize(get_data_from_HKMA(MBC))

#Convert dataframe to dataframe long format
mbc_dfl = pd.melt(mbc_df, id_vars = ['end_of_month'], value_vars =['cert_of_indebt', 'gov_notes_coins_circulation' , 'aggr_balance_bf_disc_win' , 'outstanding_efbn' , 'ow_lb_bf_disc_win' , 'mb_bf_disc_win_total'],var_name='Size of the Monetary Base', value_name='HK$ million' )

fig = px.area(mbc_dfl, x='end_of_month' , y='HK$ million', color = 'Size of the Monetary Base')

fig.show()


# 1. Money Multiplier = M3 / MB (HKD)

In [3]:
# ---------------- 1. Money Multiplier = M3 / MB (HKD) ----------------

MSA = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/money/supply-adjusted?choose=end_of_month&from=1998-09&to=now&sortby=end_of_month&sortorder=asc'

#1. There is a break in data series due to the inclusion of short-term Exchange Fund placements of less than one month since April 1997.
msa_df = pd.json_normalize(get_data_from_HKMA(MSA))


In [4]:
m3_hkd = msa_df[['end_of_month','m3_total']]

mb_tot = mbc_df[['end_of_month','mb_bf_disc_win_total']]

m3_hkd_Over_mb_tot = pd.merge(m3_hkd, mb_tot, on='end_of_month')

m3_hkd_Over_mb_tot['m3Overmb'] = (m3_hkd_Over_mb_tot['m3_total'] / m3_hkd_Over_mb_tot['mb_bf_disc_win_total'])


fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3Overmb",
              
              labels={
                     "x": "End_of_month",
                     "y": "Sepal Width (cm)"
                 },
              title='m3_hkd_Over_mb_tot')
fig.show()



In [5]:
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "m3_total",
              
              labels={
                     "x": "End_of_month",
                     "y": "m3_total"
                 },
              title='m3_total')
fig.show()

In [6]:
fig = px.line(m3_hkd_Over_mb_tot, x="end_of_month", y = "mb_bf_disc_win_total",
              
              labels={
                     "x": "End_of_month",
                     "y": "mb_bf_disc_win_total"
                 },
              title='mb_bf_disc_win_total')
fig.show()

# 2. Total deposits & M3 (HKD)

In [7]:
TD = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/customer-deposits-by-currency?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

td_records_df = pd.json_normalize(get_data_from_HKMA(TD))

deposits_all = td_records_df[['end_of_month','deposits_all']]

m3_hkd_and_deposits_all  = pd.merge(m3_hkd, deposits_all, on='end_of_month')
#Convert dataframe to dataframe long format
m3_hkd_and_deposits_all_dfl = pd.melt(m3_hkd_and_deposits_all, id_vars = ['end_of_month'], value_vars =['m3_total', 'deposits_all'],var_name='var', value_name='HK$ million' )

fig = px.area(m3_hkd_and_deposits_all_dfl, x='end_of_month' , y='HK$ million', color = 'var')
fig.show()

# 3. Loan to Deposits


In [8]:
BS = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/banking/balance-sheet-ais?choose=end_of_month&from=1998-09&to=no&sortby=end_of_month&sortorder=asc'

bs_records_df = pd.json_normalize(get_data_from_HKMA(BS))





In [9]:
bs_records_df['LtoD_hkd'] = (bs_records_df['asst_loanadvtocust_hkd'] / bs_records_df['liab_depcust_hkd'])

fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_hkd",
              title='LtoD_hkd')
fig.show()

In [10]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_hkd', 'liab_depcust_hkd'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.show()

## 3.2. Loan to Deposits (FC)


In [11]:

bs_records_df['LtoD_fc'] = (bs_records_df['asst_loanadvtocust_fc'] / bs_records_df['liab_depcust_fc'])

fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_hkd",
              title='LtoD_hkd')
fig.show()




In [12]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_fc', 'liab_depcust_fc'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.show()

## 3.3 Loan to Deposits (Total)

In [13]:
bs_records_df['LtoD_total'] = (bs_records_df['asst_loanadvtocust_total'] / bs_records_df['liab_depcust_total'])

fig = px.line(bs_records_df, x="end_of_month", y = "LtoD_hkd",
              title='LtoD_hkd')
fig.show()

In [14]:
#Convert dataframe to dataframe long format
bs_records_dfl = pd.melt(bs_records_df, id_vars = ['end_of_month'], value_vars =['asst_loanadvtocust_total', 'liab_depcust_total'],var_name='var', value_name='HK$ million' )

fig = px.line(bs_records_dfl, x='end_of_month' , y='HK$ million', color = 'var')

fig.show()

#4.Centaline index

#5.MB / AB vs HIBOR

In [45]:
HIBOR = 'https://api.hkma.gov.hk/public/market-data-and-statistics/monthly-statistical-bulletin/er-ir/hk-interbank-ir-endperiod?sortby=end_of_month&sortorder=asc'
HIBOR_df = pd.json_normalize(get_data_from_HKMA(HIBOR))

fig = px.line(HIBOR_df, x="end_of_month", y = "ir_1m",
              title='HIBOR: ir_1m')
fig.show()


#6. FPS